# Gettable and Settable


Parameters in Quantify must satisfy certain requirements as encapsulated by the Settable and Gettable classes, the specific interfaces of which are defined in the API Reference. 

The QCodes Parameter classes naturally fit the Settable and Gettable concepts; oftentimes these will be sufficient and can save development time in their reuse. In this tutorial, we will explore creating custom Gettables and Settables for more advanced experiments.

In [1]:
import numpy as np 
from quantify.measurement.types import Settable, Gettable

## Required Attributes
Let's say we want to sweep a sine wave. For this we define a pair of simple parameters fulfilling the required attributes.
Note that the set() method accepts a single float value the get() method returns either a single float value or an array-like of float values.

In [2]:
class Sweep():
    def __init__(self):
        self.unit = 'Hz'
        self.label = 'Frequency'
        self.name = 'sweep'
        self.current_val = None

    def set(setpoint):
        self.current_val = setpoint


class SinWave():
    def __init__(self, sweep):
        self.unit = 'A'
        self.label = 'Amplitude'
        self.name = 'wave'
        self.sweep = sweep

    def get(self):
        return sin(self.sweep.current_val)


sweep_range = Settable(Sweep())
sin_wave = Gettable(SinWave(sweep_range))

In [3]:
# we can see sweep_range is still of type Sweep
type(sweep_range)

__main__.Sweep

But what happens if our custom parameter is lacking a required attribute? Let's delete the `name` attribute and check:

In [4]:
del sin_wave.name
try:
    Gettable(sin_wave)
except Exception as e:
    print(e)

<__main__.SinWave object at 0x7f84067b3af0> does not have 'name'


The Gettable throws an exception warning us what part of the interface is broken.

## Control, Prepare and Finish
The prepare() and finish() functions are useful for performing work before each iteration of the measurement loop and once after completion. For example, arming a piece of hardware with data and then closing a connection upon completion. 

In [5]:
def connect_to_device():
    # connect to our hardware
    pass


class Hard(SinWave):
    def __init__(self):
        super().__init__(None)
        self.soft = False
        self.device = connect_to_device()

    def prepare(setpoints):
        self.device.arm(setpoints)

    def finish():
        self.device.close()

    def get():
        return self.device.run()


hard_sin = Gettable(Hard())